<a href="https://colab.research.google.com/github/phibr0/facharbeit/blob/main/code/Facharbeit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trainieren des Beispiel NN's aus Kapitel 2

In diesem Teil wird das Beispiel NN aus Kapitel 2 meiner Facharbeit trainiert.
Das MNIST Dataset enthält Bilder von Handgeschriebenen Ziffern mit einer Größe von 28x28 Pixeln.

**Der Code kann direkt hier im Browser ausgeführt und auch verändert werden.**

Das trainierte Modell wird anschließend als 'beispielModel_MNIST' gespeichert.

In [ ]:
#@title Variablen { form-width: "30%", display-mode: "both" }
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np

numOfNeuronsIn1 = 16 #@param {type:"slider", min:1, max:256, step:1}
numOfNeuronsIn2 = 16 #@param {type:"slider", min:1, max:256, step:1}
numOfEpochs = 5 #@param {type:"slider", min:1, max:256, step:1}

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data() # Laden des MNIST Datasets
# Und aufteilen in Trainigsdaten und Testdaten

x_train = tf.keras.utils.normalize(x_train, axis=1) # Normalisieren des Datasets
x_test = tf.keras.utils.normalize(x_test, axis=1)

model = tf.keras.models.Sequential() ## Erstellen des Neuronalen Netzwerks
model.add(layers.Flatten())
model.add(layers.Dense(numOfNeuronsIn1, activation=tf.nn.sigmoid)) # Hinzufügen der Layer
model.add(layers.Dense(numOfNeuronsIn2, activation=tf.nn.sigmoid))
model.add(layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) # Kompilieren der Layer zu einem trainierfähigen Modell

model.fit(x_train, y_train, epochs=numOfEpochs) # Trainieren des Modells mit den Trainingsdaten und x Epochen

model.save('beispielModel_MNIST') # Speichern des Modells

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.6964 - accuracy: 0.5251
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5897 - accuracy: 0.8444
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3665 - accuracy: 0.9057
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2836 - accuracy: 0.9233
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2495 - accuracy: 0.9290
INFO:tensorflow:Assets written to: beispielModel_MNIST/assets
